In [ ]:
let isBig (cave: string) = Char.IsUpper cave[0]


let parseGraph (lines: string seq) = 
  lines
  |> Seq.map (fun s -> s.Split('-'))
  |> Seq.collect (function 
    | [|"start"; y|] -> ["start",y] 
    | [|x; "start"|] -> ["start",x] 
    | [|x;y|] -> [(x,y); (y,x)])
  |> Seq.groupBy fst
  |> Map
  |> Map.map (fun k v -> v |> Seq.map snd |> Seq.toList |> List.sort)

let example = 
  @"start-A
start-b
A-c
A-b
b-d
A-end
b-end".Split('\n') |> parseGraph

let example2 = 
  @"dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc".Split('\n') |> parseGraph

let example3 = 
  @"fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW".Split('\n') |> parseGraph

let input = File.ReadAllLines "input.txt" |> parseGraph

In [ ]:
let (|Small|Big|) s = if isBig s then Big s else Small s

let removeCave cave graph =
  graph
  |> Map.remove cave
  |> Map.map (fun _ v -> List.except [cave] v)

let removeIfSmall cave graph =
  match cave with
  | Big cave -> graph
  | Small cave -> removeCave cave graph

let rec pathsFrom graph = function
  | "end" -> [["end"]]
  | cave -> 
    match Map.tryFind cave graph with
    | Some caves -> 
      caves
      |> List.collect (pathsFrom (removeIfSmall cave graph))
      |> List.map (fun path -> cave :: path)
    | None -> []

pathsFrom graph "start"
|> Seq.length

4885

In [ ]:
let paths2 graph start = 
  let addSmall smalls = function
  | Big cave -> smalls
  | Small cave -> smalls |> Set.add cave
  
  let rec p2 visitedSmalls = function
  | "end" -> [["end"]]
  | cave -> 
    if Set.contains cave visitedSmalls then 
      let remaining = 
        visitedSmalls
        |> Seq.fold (fun g s -> removeCave s g) graph
      Map.find cave graph
      |> List.collect (pathsFrom remaining)
      |> List.map(fun path -> cave :: path)
    else
      Map.find cave graph 
      |> List.collect (p2 (addSmall visitedSmalls cave))
      |> List.map(fun path -> cave :: path)
  p2 Set.empty start

paths2 input "start" |> Seq.length

117095